# 2018.12.10由于很多新建的特征，对于Test并不适用，所以在V2.1上进行进一步整理

In [95]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import datetime
import gc
gc.collect()

28

In [96]:
import lightgbm as lgb

In [97]:
flowdf=pd.read_csv('../data/flow_train.csv')
trandf=pd.read_csv('../data/transition_train.csv')

In [98]:
start='20180301'
dates=[]
for i in range(1,16):
    dd=datetime.datetime.strptime(start,"%Y%m%d")
    date=dd+datetime.timedelta(days=i)
    dates.append(int(datetime.datetime.strftime(date,"%Y%m%d")))
print(dates)

[20180302, 20180303, 20180304, 20180305, 20180306, 20180307, 20180308, 20180309, 20180310, 20180311, 20180312, 20180313, 20180314, 20180315, 20180316]


In [99]:
trandf['start_address']=trandf['o_city_code'].astype(str)+':'+trandf['o_district_code'].astype(str)
trandf['end_address']=trandf['d_city_code'].astype(str)+':'+trandf['d_district_code'].astype(str)

flowdf['address']=flowdf['city_code'].astype(str)+':'+flowdf['district_code'].astype(str)

In [100]:
start_address=list(set(trandf['start_address']))
end_address=list(set(trandf['end_address']))

### ==========没办法，得先建立个tran_train.testdf，否则脑子转不动====

In [101]:
def get_df(k=98,add_list=start_address):
    df=pd.DataFrame()
    df['row_id']=np.arange(9604)
    df['start_address']='0'
    df['end_address']='0'
    for i in range(0,9603,98):
        df['start_address'][i:(i+k)]=add_list[int(i/k)]
        for index in range(i,i+k):
            df.loc[index,'end_address']=add_list[int(index%k)]
    return df

test=get_df(k=98,add_list=start_address)
test['date_dt']=dates[0]

for i in range(1,15,1):
    df=get_df(k=98,add_list=start_address)
    df['date_dt']=dates[i]
    test=pd.concat([test,df],axis=0)
    del df
    gc.collect()

test=test[test['start_address']!=test['end_address']]
print(test.shape)

del test['row_id']

test['o_city_code']=test['start_address'].apply(lambda x:str(x).split(':')[0])
test['o_district_code']=test['start_address'].apply(lambda x:str(x).split(':')[1])
test['d_city_code']=test['end_address'].apply(lambda x:str(x).split(':')[0])
test['d_district_code']=test['end_address'].apply(lambda x:str(x).split(':')[1])
print(test.shape,test.columns)

(142590, 4)
(142590, 7) Index(['start_address', 'end_address', 'date_dt', 'o_city_code',
       'o_district_code', 'd_city_code', 'd_district_code'],
      dtype='object')


# ======哦哟，以上终于建立好了tran_train.Testdf========

### <font color=red>=====索性flow_train.testdf一起建立好了<font>

In [102]:
# print(flowdf.shape,flowdf.columns)

# dates=[20180302, 20180303, 20180304, 20180305, 20180306, 20180307, 20180308, 20180309, 20180310, 
#        20180311, 20180312, 20180313, 20180314, 20180315, 20180316]

# n=98
# flow_test=pd.DataFrame()
# flow_test['row_id']=np.arange(0,1470)
# flow_test['date_dt']=0
# for i in range(15):     
#     flow_test['date_dt'][i*n:(i+1)*n]=dates[i]
#     flow_test['address']=0
#     for j in range(0,1470):
#         flow_test.loc[flow_test['row_id']==j,'address']=start_address[j%n]
# flow_test['city_code']=flow_test['address'].apply(lambda x:str(x).split(':')[0])
# flow_test['district_code']=flow_test['address'].apply(lambda x:str(x).split(':')[1])
# print(flow_test.groupby('date_dt').address.count())

# del flow_test['row_id']

# print(flow_test.shape,flow_test.columns)

# flow_test.to_csv('../data/flow_test.csv',index=False)

# <font color=green>特征工程<font>

In [103]:
from datetime import datetime

In [104]:
def month(date):
    date=str(date)
    return int(date[4:6])

def day(date):
    date=str(date)
    return int(date[6:8])

def weekd(date):
    date=str(date)
    return str(datetime.strptime(date,'%Y%m%d').weekday())

In [105]:
gc.collect()

250

In [106]:
flow_test=pd.read_csv('../data/flow_test.csv')

In [107]:
trandf['month']=trandf['date_dt'].apply(month)
trandf['month']=trandf['month'].apply(lambda x:int(x+12) if x <4 else x)
trandf['day']=trandf['date_dt'].apply(day)
trandf['weekd']=trandf['date_dt'].apply(weekd)

test['month']=test['date_dt'].apply(month)
test['month']=test['month'].apply(lambda x:int(x+12) if x<4 else x)
test['day']=test['date_dt'].apply(day)
test['weekd']=test['date_dt'].apply(weekd)

flowdf['month']=flowdf['date_dt'].apply(month)
flowdf['month']=flowdf['month'].apply(lambda x:int(x+12) if x <4 else x)
flowdf['day']=flowdf['date_dt'].apply(day)
flowdf['weekd']=flowdf['date_dt'].apply(weekd)

flow_test['month']=flow_test['date_dt'].apply(month)
flow_test['month']=flow_test['month'].apply(lambda x:int(x+12) if x <4 else x)
flow_test['day']=flow_test['date_dt'].apply(day)
flow_test['weekd']=flow_test['date_dt'].apply(weekd)

# ========数据合并后一起处理=======

In [108]:
data=pd.concat([trandf,test],axis=0)
data['cnt']=round(data['cnt'],6)

In [109]:
#标签数值化
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

le.fit(data['o_city_code'])
data['o_city_code']=le.transform(data['o_city_code'])
data['d_city_code']=le.transform(data['d_city_code'])

le.fit(data['o_district_code'])
data['o_district_code']=le.transform(data['o_district_code'])
data['d_district_code']=le.transform(data['d_district_code'])

le.fit(data['start_address'])
data['start_address']=le.transform(data['start_address'])
flowdf['address']=le.transform(flowdf['address'])
flow_test['address_code']=flow_test['address']
flow_test['address_code']=le.transform(flow_test['address_code'])
data['end_address']=le.transform(data['end_address'])

flow_test.to_csv('../data/flow_test1.csv',index=False)
flowdf.to_csv('../data/flow_train1.csv',index=False)

In [110]:
gc.collect()

98

# <font color=red>基本特征<font>

### <font color=blue>mean  ·  nunique ·  count特征<font>

In [122]:
# print(data[data['cnt'].notnull()][['day','cnt']].groupby('day').cnt.mean().reset_index(name='notnull_mean'))

    day  notnull_mean
0     1      0.857565
1     2      0.873446
2     3      0.865991
3     4      0.854906
4     5      0.847746
5     6      0.836793
6     7      0.844189
7     8      0.836445
8     9      0.840161
9    10      0.846947
10   11      0.845310
11   12      0.835038
12   13      0.832419
13   14      0.858050
14   15      0.844272
15   16      0.834408
16   17      0.820314
17   18      0.840883
18   19      0.845215
19   20      0.852713
20   21      0.856026
21   22      0.863342
22   23      0.849249
23   24      0.867202
24   25      0.851493
25   26      0.842167
26   27      0.849426
27   28      0.870810
28   29      0.863141
29   30      0.888666
30   31      0.856422


In [123]:
# print(data[['day','cnt']].groupby('day').cnt.mean().reset_index(name='alldf_mean'))

    day  alldf_mean
0     1    0.857565
1     2    0.873446
2     3    0.865991
3     4    0.854906
4     5    0.847746
5     6    0.836793
6     7    0.844189
7     8    0.836445
8     9    0.840161
9    10    0.846947
10   11    0.845310
11   12    0.835038
12   13    0.832419
13   14    0.858050
14   15    0.844272
15   16    0.834408
16   17    0.820314
17   18    0.840883
18   19    0.845215
19   20    0.852713
20   21    0.856026
21   22    0.863342
22   23    0.849249
23   24    0.867202
24   25    0.851493
25   26    0.842167
26   27    0.849426
27   28    0.870810
28   29    0.863141
29   30    0.888666
30   31    0.856422


In [124]:
# print(data[data['cnt'].notnull()][['day','cnt']].groupby('day').cnt.count().reset_index(name='notnull_cnt'))

    day  notnull_cnt
0     1        90246
1     2        81291
2     3        81732
3     4        81587
4     5        81387
5     6        80876
6     7        80969
7     8        81426
8     9        81457
9    10        81487
10   11        81304
11   12        81159
12   13        81322
13   14        81422
14   15        81188
15   16        80871
16   17        80863
17   18        81365
18   19        81536
19   20        81521
20   21        81824
21   22        82169
22   23        81578
23   24        81783
24   25        81669
25   26        81571
26   27        81536
27   28        81763
28   29        72747
29   30        73105
30   31        45566


In [125]:
# print(data[['day','cnt']].groupby('day').cnt.count().reset_index(name='alldf_count'))

    day  alldf_count
0     1        90246
1     2        81291
2     3        81732
3     4        81587
4     5        81387
5     6        80876
6     7        80969
7     8        81426
8     9        81457
9    10        81487
10   11        81304
11   12        81159
12   13        81322
13   14        81422
14   15        81188
15   16        80871
16   17        80863
17   18        81365
18   19        81536
19   20        81521
20   21        81824
21   22        82169
22   23        81578
23   24        81783
24   25        81669
25   26        81571
26   27        81536
27   28        81763
28   29        72747
29   30        73105
30   31        45566


# <font color=red>当test中的cnt没有用0填补的时候，计算cnt的mean()或count()都不会将test的计入<font>

In [111]:
timefea=['day','weekd']
addfea=['start_address','end_address']

#时间一维
for tf in timefea:
    temp=data[[tf,'cnt']]
    meandf=temp.groupby(tf).cnt.mean().reset_index(name=tf+'_cnt_mean')
    cntdf1=temp.groupby(tf).cnt.count().reset_index(name=tf+'_cnt_count')
    data=data.merge(meandf,on=tf,how='left')
    data=data.merge(cntdf1,on=tf,how='left')
print(data.shape)
gc.collect()

(2622910, 15)


179

In [112]:
#地址一维
for af in addfea:
    temp=data[[af,'cnt']]
    meandf=temp.groupby(af).cnt.mean().reset_index(name=af+'_cnt_mean')
    cntdf=temp.groupby(af).cnt.count().reset_index(name=af+'_cnt_count')
    data=data.merge(meandf,on=af,how='left')
    data=data.merge(cntdf,on=af,how='left')

print(data.shape)
gc.collect()

(2622910, 19)


155

In [113]:
#时间二维
temp=data[['cnt','day','weekd']]
meandf=temp.groupby(['day','weekd']).cnt.mean().reset_index(name='day_week_cnt_mean')
cntdf1=temp.groupby(['day','weekd']).cnt.count().reset_index(name='day_week__cnt_count')
data=data.merge(meandf,on=['day','weekd'],how='left')
data=data.merge(cntdf1,on=['day','weekd'],how='left')
print(data.shape)
gc.collect()

(2622910, 21)


68

In [114]:
#地址二维
temp=data[['cnt','start_address','end_address']]
meandf=temp.groupby(['start_address','end_address']).cnt.mean().reset_index(name='start_end_cnt_mean')
cntdf=temp.groupby(['start_address','end_address']).cnt.count().reset_index(name='start_end_cnt_count')
data=data.merge(meandf,on=['start_address','end_address'],how='left')
data=data.merge(cntdf,on=['start_address','end_address'],how='left')
print(data.shape)
gc.collect()

(2622910, 23)


68

In [115]:
# #时间地址交叉维度
for tf in timefea:
    for af in addfea:
        temp=data[[tf,af,'cnt']]
        meandf=temp.groupby([tf,af]).cnt.mean().reset_index(name=tf+'_'+af+'_cnt_mean')
        cntdf=temp.groupby([tf,af]).cnt.count().reset_index(name=tf+'_'+af+'_cnt_count')
        data=data.merge(meandf,on=[tf,af],how='left')
        data=data.merge(cntdf,on=[tf,af],how='left')
print(data.shape)
gc.collect()

(2622910, 31)


222

In [116]:
#时间二维与单地址的交叉
for af in addfea:
    temp=data[['day','weekd',af,'cnt']]
    meandf=temp.groupby(['day','weekd',af]).cnt.mean().reset_index(name='day_week_'+af+'_cnt_mean')
    cntdf=temp.groupby(['day','weekd',af]).cnt.count().reset_index(name='day_week_'+af+'_cnt_count')
    data=data.merge(meandf,on=['day','weekd',af],how='left')
    data=data.merge(cntdf,on=['day','weekd',af],how='left')
print(data.shape)

(2622910, 35)


In [118]:
#时间一维与地址二维度的交叉
for tf in timefea:
    temp=data[['start_address','end_address',tf,'cnt']]
    meandf=temp.groupby(['start_address','end_address',tf]).cnt.mean().reset_index(name='st_end_'+tf+'_cnt_mean')
    cntdf=temp.groupby(['start_address','end_address',tf]).cnt.count().reset_index(name='st_end_'+tf+'_cnt_count')
    data=data.merge(meandf,on=['start_address','end_address',tf],how='left')
    data=data.merge(cntdf,on=['start_address','end_address',tf],how='left')
print(data.shape)

(2622910, 39)


In [119]:
del temp
del meandf
del cntdf
gc.collect()

408

# <font color=red>由于因变量y值呈偏右态长尾分布，其中17%的数据的y值均大于1的，相对于0.0003类的数据，1.0已然偏<font>

In [120]:
data['cnt'][data['cnt'].notnull()]=data['cnt'][data['cnt'].notnull()].apply(lambda x:round(np.sqrt(x),6))

In [121]:
print(pd.isnull(data).sum())

cnt                                 142590
d_city_code                              0
d_district_code                          0
date_dt                                  0
day                                      0
end_address                              0
month                                    0
o_city_code                              0
o_district_code                          0
start_address                            0
weekd                                    0
day_cnt_mean                             0
day_cnt_count                            0
weekd_cnt_mean                           0
weekd_cnt_count                          0
start_address_cnt_mean                   0
start_address_cnt_count                  0
end_address_cnt_mean                     0
end_address_cnt_count                    0
day_week_cnt_mean                        0
day_week__cnt_count                      0
start_end_cnt_mean                       0
start_end_cnt_count                      0
day_start_a

# <font color=red>以上几个缺失值，再用lightgbm的时候，是可以不用补上的，算法自动处理；如果用其他算法的时候，建议直接0补全<font>

### 历史转移率特征，来挖掘历史转移信息，同时防止过拟合（参考历史点击率特征）

In [126]:
# data['period'] = data['day'].astype(int)
# for feat_1 in addfea:
#     res = pd.DataFrame()
#     temp = data[data['cnt']!=0][[feat_1, 'period', 'cnt']]
#     for period in range(1, 32):
#         if period == 1:
#             count = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=feat_1 + '_all')
#             count1 = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=feat_1 + '_1')
#         else:
#             count = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=feat_1 + '_all')
#             count1 = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=feat_1 + '_1')
#         count[feat_1 + '_1'] = count1[feat_1 + '_1']
#         count.fillna(value=0, inplace=True)
#         count[feat_1 + '_rate'] = round(count[feat_1 + '_1'] / count[feat_1 + '_all'], 6)
#         count['period'] = period
#         count.drop([feat_1 + '_all', feat_1 + '_1'], axis=1, inplace=True)
#         count.fillna(value=0, inplace=True)
#         res = res.append(count, ignore_index=True)
#     print(feat_1, ' over')
#     data = pd.merge(data, res, how='left', on=[feat_1, 'period'])
# print(data.shape)
# gc.collect()

In [127]:
# data['period'] = data['day']
# feat_1=['start_address','end_address']
# res = pd.DataFrame()
# temp = data[data['cnt']!=0][['start_address','end_address','period', 'cnt']]
# for period in range(1, 32):
#     if period == 1:
#         count = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=str(period)+'_all')
#         count1 = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=str(period)+'_1')
#     else:
#         count = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=str(period)+'_all')
#         count1 = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=str(period)+'_1')
#     count[str(period)+'_1'] = count1[str(period)+'_1']
#     count.fillna(value=0, inplace=True)
#     count[str(period)+'_rate'] = round(count[str(period)+'_1'] / count[str(period)+'_all'], 6)
#     count['period'] = period
#     count.drop([str(period)+'_all', str(period)+'_1'], axis=1, inplace=True)
#     count.fillna(value=0, inplace=True)
#     res = res.append(count, ignore_index=True)
# data = pd.merge(data, res, how='left', on=['start_address','end_address', 'period'])
# print(data.shape)
# gc.collect()

In [128]:
# data['period'] = data['weekd'].astype(int)
# for feat_1 in addfea:
#     res = pd.DataFrame()
#     temp = data[data['cnt']!=0][[feat_1, 'period', 'cnt']]
#     for period in range(0, 7):
#         if period == 0:
#             count = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=feat_1 + '_all')
#             count1 = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=feat_1 + '_1')
#         else:
#             count = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=feat_1 + '_all')
#             count1 = temp.groupby([feat_1]).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=feat_1 + '_1')
#         count[feat_1 + '_1'] = count1[feat_1 + '_1']
#         count.fillna(value=0, inplace=True)
#         count[feat_1 + '_rate'] = round(count[feat_1 + '_1'] / count[feat_1 + '_all'], 6)
#         count['period'] = period
#         count.drop([feat_1 + '_all', feat_1 + '_1'], axis=1, inplace=True)
#         count.fillna(value=0, inplace=True)
#         res = res.append(count, ignore_index=True)
#     print(feat_1, ' over')
#     data = pd.merge(data, res, how='left', on=[feat_1, 'period'])
# print(data.shape)
# gc.collect()

In [129]:
# data['period'] = data['weekd'].astype(int)
# feat_1=['start_address','end_address']
# res = pd.DataFrame()
# temp = data[data['cnt']!=0][['start_address','end_address','period', 'cnt']]
# for period in range(0, 7):
#     if period == 0:
#         count = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=str(period)+'_all')
#         count1 = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=str(period)+'_1')
#     else:
#         count = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=str(period)+'_all')
#         count1 = temp.groupby(feat_1).apply(
#                 lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=str(period)+'_1')
#     count[str(period)+'_1'] = count1[str(period)+'_1']
#     count.fillna(value=0, inplace=True)
#     count[str(period)+'_rate'] = round(count[str(period)+'_1'] / count[str(period)+'_all'], 6)
#     count['period'] = period
#     count.drop([str(period)+'_all', str(period)+'_1'], axis=1, inplace=True)
#     count.fillna(value=0, inplace=True)
#     res = res.append(count, ignore_index=True)
# data = pd.merge(data, res, how='left', on=['start_address','end_address', 'period'])
# print(data.shape)
# gc.collect()

# =========数据本地保存========

In [130]:
# ======按照出发城市进行分集存储后提取训练==============

#按照出发城市进行分集存储
def save_data(path,data):
    for i in range(0,7):
        ndata=data[data['o_city_code']==i]
        ndata.to_csv(path+'data_'+str(i)+'.csv',index=False)
        
path='../data/'
save_data(path,data)

gc.collect()

17

In [52]:
#用来调最优参数的自定义函数
def get_best_lgb(path,i,N):
    data=pd.read_csv(path+'data_'+str(i)+'.csv')
    traindf=data[data['month']!=15]
    testdf=data[data['month']==15]
    print(traindf.shape,testdf.shape)
    
    train_xy,off_xy=train_test_split(traindf,test_size=0.1,random_state=1)
    tr,val=train_test_split(train_xy,test_size=0.15,random_state=1)
    
    drop_list=['cnt','o_city_code','o_district_code','date_dt','month']
    y_train=tr.cnt
    x_train=tr.drop(drop_list,axis=1)
    
    y_val=val.cnt
    x_val=val.drop(drop_list,axis=1)
    
    y_off=off_xy.cnt
    x_off=off_xy.drop(drop_list,axis=1)
    
    test_x=testdf.drop(drop_list,axis=1)
    
    gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=70,
                        learning_rate=0.01,
                        n_estimators=N)

    gbm.fit(x_train, y_train,
        eval_set=[(x_train,y_train),(x_val,y_val)],
        eval_metric='l1',
        early_stopping_rounds=100)

    y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
    y_eval=rmsle(y_val,y_pred)[1]
    print('The rmlse of prediction is:',y_eval)
    print('Feature importances:', list(gbm.feature_importances_))

    y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
    y_off_eval=rmsle(y_off, y_pred_off)[1]
    print('The rmlse of prediction is:',y_off_eval)

    gc.collect()

In [ ]:
def get_lgb(path,i,N):
    data=pd.read_csv(path+'data_'+str(i)+'.csv')
    traindf=data[data['month']!=15]
    testdf=data[data['month']==15]
    print(traindf.shape,testdf.shape)
    
    train_xy,off_xy=train_test_split(traindf,test_size=0.1,random_state=1)
    tr,val=train_test_split(train_xy,test_size=0.15,random_state=1)
    
    drop_list=['cnt','o_city_code','date_dt']
    
    #用于后面的数据融合
    y_traindf=traindf.cnt
    x_traindf=traindf.drop(drop_list,axis=1)
    
    #将traindf分成训练和线下验证二种，训练集又分成训练和验证二种
    y_train=tr.cnt
    x_train=tr.drop(drop_list,axis=1)
    
    y_val=val.cnt
    x_val=val.drop(drop_list,axis=1)
    
    y_off=off_xy.cnt
    x_off=off_xy.drop(drop_list,axis=1)
    
    test_x=testdf.drop(drop_list,axis=1)
    
    gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=50,
                        learning_rate=0.05,
                        n_estimators=N)

    gbm.fit(x_train, y_train,
        eval_set=[(x_train,y_train),(x_val,y_val)],
        eval_metric='l1',
        early_stopping_rounds=50)

    y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
    y_eval=rmsle(y_val,y_pred)[1]
    # print('The rmse of prediction is:', mean_squared_error(y_val, y_pred) ** 0.5)
    print('The rmlse of prediction is:',y_eval)
    # print('Feature importances:', list(gbm.feature_importances_))

    y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
    y_off_eval=rmsle(y_off, y_pred_off)[1]
    # print('The rmse of prediction is:', mean_squared_error(y_off, y_pred_off) ** 0.5)
    print('The rmlse of prediction is:',y_off_eval)

    gc.collect()
    
    traindf['cnt_p']=gbm.predict(x_traindf,num_iteration=gbm.best_iteration_)
    testdf['cnt_p']=gbm.predict(test_x,num_iteration=gbm.best_iteration_)
    
    datadf=pd.concat([traindf,testdf],axis=0)
    datadf[['date_dt','start_address','end_address','cnt','cnt_p']].to_csv(path+'dataf_'+str(i)+'.csv',index=False)
    
    return y_eval,y_off_eval

In [39]:
rmsle0=get_best_lgb(path,0,N=942) #it is ok!
#     gbm = lgb.LGBMRegressor(objective='regression',
#                         num_leaves=50,
#                         learning_rate=0.05,
#                         n_estimators=N)

#     gbm.fit(x_train, y_train,
#         eval_set=[(x_train,y_train),(x_val,y_val)],
#         eval_metric='l1',
#         early_stopping_rounds=100)

(273015, 36) (16980, 36)
[1]	training's l1: 1.11249	valid_1's l1: 1.14016
Training until validation scores don't improve for 50 rounds.
[2]	training's l1: 1.05753	valid_1's l1: 1.0837
[3]	training's l1: 1.00538	valid_1's l1: 1.03005
[4]	training's l1: 0.95589	valid_1's l1: 0.97916
[5]	training's l1: 0.908918	valid_1's l1: 0.9308
[6]	training's l1: 0.864299	valid_1's l1: 0.884972
[7]	training's l1: 0.821913	valid_1's l1: 0.841365
[8]	training's l1: 0.781668	valid_1's l1: 0.800018
[9]	training's l1: 0.74348	valid_1's l1: 0.76075
[10]	training's l1: 0.707203	valid_1's l1: 0.723458
[11]	training's l1: 0.672789	valid_1's l1: 0.688095
[12]	training's l1: 0.640098	valid_1's l1: 0.654498
[13]	training's l1: 0.609091	valid_1's l1: 0.622677
[14]	training's l1: 0.579694	valid_1's l1: 0.592482
[15]	training's l1: 0.551782	valid_1's l1: 0.563794
[16]	training's l1: 0.525367	valid_1's l1: 0.536614
[17]	training's l1: 0.500339	valid_1's l1: 0.5109
[18]	training's l1: 0.476615	valid_1's l1: 0.486602
[

[156]	training's l1: 0.0705996	valid_1's l1: 0.0848102
[157]	training's l1: 0.0705508	valid_1's l1: 0.0848075
[158]	training's l1: 0.0705178	valid_1's l1: 0.0848279
[159]	training's l1: 0.0704832	valid_1's l1: 0.0848538
[160]	training's l1: 0.0704374	valid_1's l1: 0.0848532
[161]	training's l1: 0.0704068	valid_1's l1: 0.0848767
[162]	training's l1: 0.0703517	valid_1's l1: 0.0848617
[163]	training's l1: 0.0703177	valid_1's l1: 0.0849019
[164]	training's l1: 0.0702922	valid_1's l1: 0.0849368
[165]	training's l1: 0.0702556	valid_1's l1: 0.084938
[166]	training's l1: 0.0702355	valid_1's l1: 0.084977
[167]	training's l1: 0.0701974	valid_1's l1: 0.0849779
[168]	training's l1: 0.0701664	valid_1's l1: 0.0849876
[169]	training's l1: 0.0701399	valid_1's l1: 0.0850084
[170]	training's l1: 0.0701145	valid_1's l1: 0.0850171
[171]	training's l1: 0.070092	valid_1's l1: 0.0850367
[172]	training's l1: 0.0700661	valid_1's l1: 0.0850511
[173]	training's l1: 0.0700303	valid_1's l1: 0.0850612
[174]	trainin

In [48]:
rmsle2=get_best_lgb(path,2,N=1000)  not ok, why?
#     gbm = lgb.LGBMRegressor(objective='regression',
#                         num_leaves=50,
#                         learning_rate=0.01,
#                         n_estimators=N)

#     gbm.fit(x_train, y_train,
#         eval_set=[(x_train,y_train),(x_val,y_val)],
#         eval_metric='l1',
#         early_stopping_rounds=100)

(256286, 36) (15500, 36)
[1]	training's l1: 3.81284	valid_1's l1: 3.7846
Training until validation scores don't improve for 50 rounds.
[2]	training's l1: 3.6253	valid_1's l1: 3.59818
[3]	training's l1: 3.44698	valid_1's l1: 3.42083
[4]	training's l1: 3.27804	valid_1's l1: 3.253
[5]	training's l1: 3.11749	valid_1's l1: 3.09329
[6]	training's l1: 2.96496	valid_1's l1: 2.94165
[7]	training's l1: 2.82041	valid_1's l1: 2.7981
[8]	training's l1: 2.68328	valid_1's l1: 2.66179
[9]	training's l1: 2.55293	valid_1's l1: 2.53232
[10]	training's l1: 2.4292	valid_1's l1: 2.40937
[11]	training's l1: 2.31189	valid_1's l1: 2.29274
[12]	training's l1: 2.20067	valid_1's l1: 2.18225
[13]	training's l1: 2.09499	valid_1's l1: 2.07719
[14]	training's l1: 1.99479	valid_1's l1: 1.97762
[15]	training's l1: 1.89975	valid_1's l1: 1.88338
[16]	training's l1: 1.80968	valid_1's l1: 1.79403
[17]	training's l1: 1.72435	valid_1's l1: 1.70941
[18]	training's l1: 1.64362	valid_1's l1: 1.62907
[19]	training's l1: 1.56714	

[159]	training's l1: 0.278473	valid_1's l1: 0.323269
[160]	training's l1: 0.278295	valid_1's l1: 0.323222
[161]	training's l1: 0.278136	valid_1's l1: 0.323193
[162]	training's l1: 0.277952	valid_1's l1: 0.323178
[163]	training's l1: 0.277849	valid_1's l1: 0.323224
[164]	training's l1: 0.277732	valid_1's l1: 0.323303
[165]	training's l1: 0.277548	valid_1's l1: 0.32322
[166]	training's l1: 0.277352	valid_1's l1: 0.323212
[167]	training's l1: 0.277202	valid_1's l1: 0.323187
[168]	training's l1: 0.27709	valid_1's l1: 0.323196
[169]	training's l1: 0.276957	valid_1's l1: 0.323179
[170]	training's l1: 0.276875	valid_1's l1: 0.32325
[171]	training's l1: 0.276734	valid_1's l1: 0.323296
[172]	training's l1: 0.276606	valid_1's l1: 0.323255
[173]	training's l1: 0.276473	valid_1's l1: 0.323236
[174]	training's l1: 0.276374	valid_1's l1: 0.323276
[175]	training's l1: 0.276286	valid_1's l1: 0.323306
[176]	training's l1: 0.276184	valid_1's l1: 0.323357
[177]	training's l1: 0.276027	valid_1's l1: 0.323

In [62]:
get_best_lgb(path,1,N=1450)  it is ok
#     gbm = lgb.LGBMRegressor(objective='regression',
#                         num_leaves=60,
#                         learning_rate=0.05,
#                         n_estimators=N)

#     gbm.fit(x_train, y_train,
#         eval_set=[(x_train,y_train),(x_val,y_val)],
#         eval_metric='l1',
#         early_stopping_rounds=100)

(567719, 36) (34104, 36)
[1]	training's l1: 1.99603	valid_1's l1: 2.02453
Training until validation scores don't improve for 100 rounds.
[2]	training's l1: 1.89746	valid_1's l1: 1.92465
[3]	training's l1: 1.80391	valid_1's l1: 1.8298
[4]	training's l1: 1.71504	valid_1's l1: 1.73977
[5]	training's l1: 1.63073	valid_1's l1: 1.65435
[6]	training's l1: 1.55059	valid_1's l1: 1.57317
[7]	training's l1: 1.47461	valid_1's l1: 1.49622
[8]	training's l1: 1.40243	valid_1's l1: 1.42308
[9]	training's l1: 1.33399	valid_1's l1: 1.35372
[10]	training's l1: 1.26898	valid_1's l1: 1.2879
[11]	training's l1: 1.20735	valid_1's l1: 1.22539
[12]	training's l1: 1.14888	valid_1's l1: 1.16613
[13]	training's l1: 1.09339	valid_1's l1: 1.10994
[14]	training's l1: 1.04084	valid_1's l1: 1.05665
[15]	training's l1: 0.990984	valid_1's l1: 1.00609
[16]	training's l1: 0.943762	valid_1's l1: 0.958171
[17]	training's l1: 0.898988	valid_1's l1: 0.91276
[18]	training's l1: 0.856513	valid_1's l1: 0.869669
[19]	training's l

[159]	training's l1: 0.127736	valid_1's l1: 0.142592
[160]	training's l1: 0.127682	valid_1's l1: 0.142612
[161]	training's l1: 0.127596	valid_1's l1: 0.142564
[162]	training's l1: 0.127547	valid_1's l1: 0.142599
[163]	training's l1: 0.127451	valid_1's l1: 0.142568
[164]	training's l1: 0.12738	valid_1's l1: 0.142546
[165]	training's l1: 0.127307	valid_1's l1: 0.142541
[166]	training's l1: 0.127251	valid_1's l1: 0.142511
[167]	training's l1: 0.127163	valid_1's l1: 0.142457
[168]	training's l1: 0.127121	valid_1's l1: 0.142458
[169]	training's l1: 0.127051	valid_1's l1: 0.142459
[170]	training's l1: 0.127014	valid_1's l1: 0.1425
[171]	training's l1: 0.126965	valid_1's l1: 0.142505
[172]	training's l1: 0.126928	valid_1's l1: 0.142511
[173]	training's l1: 0.126856	valid_1's l1: 0.142509
[174]	training's l1: 0.126807	valid_1's l1: 0.1425
[175]	training's l1: 0.126719	valid_1's l1: 0.142481
[176]	training's l1: 0.126645	valid_1's l1: 0.142447
[177]	training's l1: 0.126607	valid_1's l1: 0.14248

In [65]:
get_best_lgb(path,5,N=1450)

(556288, 36) (34044, 36)
[1]	training's l1: 4.1637	valid_1's l1: 4.24108
Training until validation scores don't improve for 100 rounds.
[2]	training's l1: 4.12262	valid_1's l1: 4.19928
[3]	training's l1: 4.08195	valid_1's l1: 4.15789
[4]	training's l1: 4.0417	valid_1's l1: 4.11694
[5]	training's l1: 4.00186	valid_1's l1: 4.07639
[6]	training's l1: 3.96241	valid_1's l1: 4.03628
[7]	training's l1: 3.92337	valid_1's l1: 3.99656
[8]	training's l1: 3.88472	valid_1's l1: 3.95725
[9]	training's l1: 3.84647	valid_1's l1: 3.91833
[10]	training's l1: 3.8086	valid_1's l1: 3.87981
[11]	training's l1: 3.77109	valid_1's l1: 3.84169
[12]	training's l1: 3.73399	valid_1's l1: 3.80394
[13]	training's l1: 3.69723	valid_1's l1: 3.76656
[14]	training's l1: 3.66085	valid_1's l1: 3.72956
[15]	training's l1: 3.62486	valid_1's l1: 3.69293
[16]	training's l1: 3.58922	valid_1's l1: 3.65667
[17]	training's l1: 3.55395	valid_1's l1: 3.62077
[18]	training's l1: 3.51903	valid_1's l1: 3.58525
[19]	training's l1: 3.48

[164]	training's l1: 0.892328	valid_1's l1: 0.910161
[165]	training's l1: 0.884745	valid_1's l1: 0.902437
[166]	training's l1: 0.877229	valid_1's l1: 0.894829
[167]	training's l1: 0.869796	valid_1's l1: 0.887279
[168]	training's l1: 0.86244	valid_1's l1: 0.879778
[169]	training's l1: 0.855192	valid_1's l1: 0.872424
[170]	training's l1: 0.848001	valid_1's l1: 0.865125
[171]	training's l1: 0.840884	valid_1's l1: 0.857923
[172]	training's l1: 0.833845	valid_1's l1: 0.850797
[173]	training's l1: 0.826872	valid_1's l1: 0.843737
[174]	training's l1: 0.819992	valid_1's l1: 0.836771
[175]	training's l1: 0.813181	valid_1's l1: 0.829861
[176]	training's l1: 0.806442	valid_1's l1: 0.823021
[177]	training's l1: 0.799785	valid_1's l1: 0.816298
[178]	training's l1: 0.793191	valid_1's l1: 0.809618
[179]	training's l1: 0.78667	valid_1's l1: 0.80301
[180]	training's l1: 0.780237	valid_1's l1: 0.796508
[181]	training's l1: 0.773866	valid_1's l1: 0.790062
[182]	training's l1: 0.767554	valid_1's l1: 0.783

[322]	training's l1: 0.328754	valid_1's l1: 0.343485
[323]	training's l1: 0.327548	valid_1's l1: 0.342319
[324]	training's l1: 0.326353	valid_1's l1: 0.341141
[325]	training's l1: 0.325161	valid_1's l1: 0.33999
[326]	training's l1: 0.323989	valid_1's l1: 0.33884
[327]	training's l1: 0.322846	valid_1's l1: 0.337721
[328]	training's l1: 0.321714	valid_1's l1: 0.336616
[329]	training's l1: 0.320581	valid_1's l1: 0.335516
[330]	training's l1: 0.31949	valid_1's l1: 0.334434
[331]	training's l1: 0.318396	valid_1's l1: 0.333387
[332]	training's l1: 0.317324	valid_1's l1: 0.332342
[333]	training's l1: 0.31626	valid_1's l1: 0.331301
[334]	training's l1: 0.315219	valid_1's l1: 0.330287
[335]	training's l1: 0.31417	valid_1's l1: 0.329267
[336]	training's l1: 0.313148	valid_1's l1: 0.328283
[337]	training's l1: 0.312142	valid_1's l1: 0.327297
[338]	training's l1: 0.311145	valid_1's l1: 0.326335
[339]	training's l1: 0.310157	valid_1's l1: 0.325382
[340]	training's l1: 0.309201	valid_1's l1: 0.32443

[478]	training's l1: 0.244655	valid_1's l1: 0.26524
[479]	training's l1: 0.24449	valid_1's l1: 0.265132
[480]	training's l1: 0.244324	valid_1's l1: 0.265006
[481]	training's l1: 0.244163	valid_1's l1: 0.264882
[482]	training's l1: 0.243999	valid_1's l1: 0.264765
[483]	training's l1: 0.243805	valid_1's l1: 0.264615
[484]	training's l1: 0.243615	valid_1's l1: 0.264461
[485]	training's l1: 0.243463	valid_1's l1: 0.26434
[486]	training's l1: 0.243275	valid_1's l1: 0.264176
[487]	training's l1: 0.243112	valid_1's l1: 0.264061
[488]	training's l1: 0.242968	valid_1's l1: 0.263955
[489]	training's l1: 0.242775	valid_1's l1: 0.263794
[490]	training's l1: 0.242619	valid_1's l1: 0.263685
[491]	training's l1: 0.242441	valid_1's l1: 0.263547
[492]	training's l1: 0.242265	valid_1's l1: 0.26339
[493]	training's l1: 0.242082	valid_1's l1: 0.26324
[494]	training's l1: 0.241945	valid_1's l1: 0.263138
[495]	training's l1: 0.241764	valid_1's l1: 0.263014
[496]	training's l1: 0.24159	valid_1's l1: 0.262867

[637]	training's l1: 0.229017	valid_1's l1: 0.255288
[638]	training's l1: 0.22894	valid_1's l1: 0.255248
[639]	training's l1: 0.228906	valid_1's l1: 0.255247
[640]	training's l1: 0.228829	valid_1's l1: 0.25521
[641]	training's l1: 0.228756	valid_1's l1: 0.255166
[642]	training's l1: 0.228707	valid_1's l1: 0.255151
[643]	training's l1: 0.228673	valid_1's l1: 0.255151
[644]	training's l1: 0.228599	valid_1's l1: 0.255111
[645]	training's l1: 0.228523	valid_1's l1: 0.255059
[646]	training's l1: 0.22849	valid_1's l1: 0.255058
[647]	training's l1: 0.228417	valid_1's l1: 0.25502
[648]	training's l1: 0.228366	valid_1's l1: 0.255013
[649]	training's l1: 0.228329	valid_1's l1: 0.255006
[650]	training's l1: 0.228243	valid_1's l1: 0.254964
[651]	training's l1: 0.228204	valid_1's l1: 0.254952
[652]	training's l1: 0.228123	valid_1's l1: 0.254915
[653]	training's l1: 0.22809	valid_1's l1: 0.254924
[654]	training's l1: 0.228053	valid_1's l1: 0.254915
[655]	training's l1: 0.228017	valid_1's l1: 0.25494

[796]	training's l1: 0.222422	valid_1's l1: 0.253686
[797]	training's l1: 0.222369	valid_1's l1: 0.253652
[798]	training's l1: 0.222352	valid_1's l1: 0.253683
[799]	training's l1: 0.222306	valid_1's l1: 0.253657
[800]	training's l1: 0.222262	valid_1's l1: 0.253629
[801]	training's l1: 0.222221	valid_1's l1: 0.253606
[802]	training's l1: 0.222203	valid_1's l1: 0.253638
[803]	training's l1: 0.222158	valid_1's l1: 0.253618
[804]	training's l1: 0.222115	valid_1's l1: 0.253596
[805]	training's l1: 0.222064	valid_1's l1: 0.253565
[806]	training's l1: 0.22202	valid_1's l1: 0.25353
[807]	training's l1: 0.221996	valid_1's l1: 0.253526
[808]	training's l1: 0.221942	valid_1's l1: 0.253502
[809]	training's l1: 0.2219	valid_1's l1: 0.253484
[810]	training's l1: 0.22185	valid_1's l1: 0.253464
[811]	training's l1: 0.221811	valid_1's l1: 0.253458
[812]	training's l1: 0.221777	valid_1's l1: 0.253438
[813]	training's l1: 0.221762	valid_1's l1: 0.253473
[814]	training's l1: 0.221714	valid_1's l1: 0.25344

[957]	training's l1: 0.217777	valid_1's l1: 0.252392
[958]	training's l1: 0.217762	valid_1's l1: 0.252391
[959]	training's l1: 0.217749	valid_1's l1: 0.252389
[960]	training's l1: 0.217726	valid_1's l1: 0.252381
[961]	training's l1: 0.217709	valid_1's l1: 0.252384
[962]	training's l1: 0.217688	valid_1's l1: 0.252379
[963]	training's l1: 0.217673	valid_1's l1: 0.252381
[964]	training's l1: 0.217646	valid_1's l1: 0.252376
[965]	training's l1: 0.217631	valid_1's l1: 0.252376
[966]	training's l1: 0.217618	valid_1's l1: 0.252378
[967]	training's l1: 0.217604	valid_1's l1: 0.252372
[968]	training's l1: 0.217584	valid_1's l1: 0.252369
[969]	training's l1: 0.217567	valid_1's l1: 0.252368
[970]	training's l1: 0.217548	valid_1's l1: 0.252373
[971]	training's l1: 0.217526	valid_1's l1: 0.252372
[972]	training's l1: 0.217508	valid_1's l1: 0.252371
[973]	training's l1: 0.217488	valid_1's l1: 0.252368
[974]	training's l1: 0.217472	valid_1's l1: 0.25237
[975]	training's l1: 0.217451	valid_1's l1: 0.2

In [66]:
get_best_lgb(path,3,N=700)
# rmsle4=get_lgb(path,4,N=700)

(405264, 36) (24748, 36)
[1]	training's l1: 2.02065	valid_1's l1: 2.018
Training until validation scores don't improve for 100 rounds.
[2]	training's l1: 2.00071	valid_1's l1: 1.9982
[3]	training's l1: 1.98098	valid_1's l1: 1.9786
[4]	training's l1: 1.96144	valid_1's l1: 1.95919
[5]	training's l1: 1.94211	valid_1's l1: 1.93999
[6]	training's l1: 1.92298	valid_1's l1: 1.92098
[7]	training's l1: 1.90404	valid_1's l1: 1.90217
[8]	training's l1: 1.88529	valid_1's l1: 1.88356
[9]	training's l1: 1.86676	valid_1's l1: 1.86514
[10]	training's l1: 1.8484	valid_1's l1: 1.84693
[11]	training's l1: 1.83023	valid_1's l1: 1.82891
[12]	training's l1: 1.81225	valid_1's l1: 1.81107
[13]	training's l1: 1.79446	valid_1's l1: 1.79342
[14]	training's l1: 1.77687	valid_1's l1: 1.77595
[15]	training's l1: 1.75945	valid_1's l1: 1.75865
[16]	training's l1: 1.7422	valid_1's l1: 1.7415
[17]	training's l1: 1.72515	valid_1's l1: 1.72455
[18]	training's l1: 1.70828	valid_1's l1: 1.7078
[19]	training's l1: 1.69157	v

[162]	training's l1: 0.466307	valid_1's l1: 0.473759
[163]	training's l1: 0.462698	valid_1's l1: 0.47018
[164]	training's l1: 0.459135	valid_1's l1: 0.466649
[165]	training's l1: 0.455604	valid_1's l1: 0.463154
[166]	training's l1: 0.452115	valid_1's l1: 0.459687
[167]	training's l1: 0.448657	valid_1's l1: 0.456264
[168]	training's l1: 0.445248	valid_1's l1: 0.452878
[169]	training's l1: 0.44187	valid_1's l1: 0.449535
[170]	training's l1: 0.438533	valid_1's l1: 0.446216
[171]	training's l1: 0.435228	valid_1's l1: 0.442932
[172]	training's l1: 0.431968	valid_1's l1: 0.439704
[173]	training's l1: 0.428737	valid_1's l1: 0.436502
[174]	training's l1: 0.425546	valid_1's l1: 0.433336
[175]	training's l1: 0.422387	valid_1's l1: 0.430207
[176]	training's l1: 0.419259	valid_1's l1: 0.427114
[177]	training's l1: 0.416169	valid_1's l1: 0.424059
[178]	training's l1: 0.41311	valid_1's l1: 0.421036
[179]	training's l1: 0.410089	valid_1's l1: 0.418044
[180]	training's l1: 0.407099	valid_1's l1: 0.415

[323]	training's l1: 0.195569	valid_1's l1: 0.209162
[324]	training's l1: 0.19499	valid_1's l1: 0.208604
[325]	training's l1: 0.194434	valid_1's l1: 0.208074
[326]	training's l1: 0.193876	valid_1's l1: 0.207549
[327]	training's l1: 0.193314	valid_1's l1: 0.207008
[328]	training's l1: 0.192774	valid_1's l1: 0.206498
[329]	training's l1: 0.19223	valid_1's l1: 0.205989
[330]	training's l1: 0.191691	valid_1's l1: 0.205486
[331]	training's l1: 0.191167	valid_1's l1: 0.205004
[332]	training's l1: 0.190651	valid_1's l1: 0.204519
[333]	training's l1: 0.190145	valid_1's l1: 0.20404
[334]	training's l1: 0.189635	valid_1's l1: 0.203557
[335]	training's l1: 0.189131	valid_1's l1: 0.203084
[336]	training's l1: 0.188641	valid_1's l1: 0.202634
[337]	training's l1: 0.188154	valid_1's l1: 0.20217
[338]	training's l1: 0.187676	valid_1's l1: 0.201727
[339]	training's l1: 0.187196	valid_1's l1: 0.201285
[340]	training's l1: 0.186729	valid_1's l1: 0.20087
[341]	training's l1: 0.186263	valid_1's l1: 0.20043

[478]	training's l1: 0.156396	valid_1's l1: 0.174847
[479]	training's l1: 0.156301	valid_1's l1: 0.174784
[480]	training's l1: 0.156221	valid_1's l1: 0.17473
[481]	training's l1: 0.156128	valid_1's l1: 0.17466
[482]	training's l1: 0.156054	valid_1's l1: 0.174622
[483]	training's l1: 0.155965	valid_1's l1: 0.174564
[484]	training's l1: 0.155892	valid_1's l1: 0.174531
[485]	training's l1: 0.1558	valid_1's l1: 0.174463
[486]	training's l1: 0.155717	valid_1's l1: 0.17441
[487]	training's l1: 0.155634	valid_1's l1: 0.174353
[488]	training's l1: 0.155566	valid_1's l1: 0.174303
[489]	training's l1: 0.155468	valid_1's l1: 0.17424
[490]	training's l1: 0.15539	valid_1's l1: 0.174186
[491]	training's l1: 0.155309	valid_1's l1: 0.174134
[492]	training's l1: 0.155243	valid_1's l1: 0.174095
[493]	training's l1: 0.155158	valid_1's l1: 0.174042
[494]	training's l1: 0.155075	valid_1's l1: 0.173987
[495]	training's l1: 0.154996	valid_1's l1: 0.173931
[496]	training's l1: 0.154928	valid_1's l1: 0.17389
[

[638]	training's l1: 0.148938	valid_1's l1: 0.171646
[639]	training's l1: 0.148905	valid_1's l1: 0.171645
[640]	training's l1: 0.148874	valid_1's l1: 0.171639
[641]	training's l1: 0.148841	valid_1's l1: 0.171632
[642]	training's l1: 0.148809	valid_1's l1: 0.171629
[643]	training's l1: 0.148777	valid_1's l1: 0.171622
[644]	training's l1: 0.148764	valid_1's l1: 0.171628
[645]	training's l1: 0.148734	valid_1's l1: 0.171626
[646]	training's l1: 0.148702	valid_1's l1: 0.171617
[647]	training's l1: 0.148683	valid_1's l1: 0.171638
[648]	training's l1: 0.148653	valid_1's l1: 0.171634
[649]	training's l1: 0.14862	valid_1's l1: 0.171623
[650]	training's l1: 0.148591	valid_1's l1: 0.171623
[651]	training's l1: 0.148576	valid_1's l1: 0.171629
[652]	training's l1: 0.14854	valid_1's l1: 0.171612
[653]	training's l1: 0.148513	valid_1's l1: 0.17161
[654]	training's l1: 0.148478	valid_1's l1: 0.171601
[655]	training's l1: 0.148461	valid_1's l1: 0.171626
[656]	training's l1: 0.148429	valid_1's l1: 0.171

In [ ]:
rmsle6=get_lgb(path,6,N=300) 